https://www.youtube.com/watch?v=06TE_U21FK4

GET THE CODE FROM THE VIDEO:
https://github.com/nicknochnack/MediaPipePoseEstimation

# 1. Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python
!pip install opencv-contrib-python==4.5.3.56
!pip install --upgrade matplotlib

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 使用影片檔案來源
cap = cv2.VideoCapture("/pathway to tour video/video")

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        # 讀取影格
        ret, frame = cap.read()
        
        # 檢查是否成功讀取到影格
        if not ret:
            print("End of video. Exiting...")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        # 顯示處理後的影像
        cv2.imshow('Mediapipe Feed', image)

        # 按 'q' 鍵退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # 釋放影片檔案來源
    cap.release()
    cv2.destroyAllWindows()


# 3. Calculate Number of Final Sit-to-Stand Movements and draw angle chart

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 使用影片檔案來源
cap = cv2.VideoCapture("/Users/peilinyang/Desktop/Sit_to_Stand/01.mp4")
# 即時影像偵測：cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    angles_list = []  # 用來存儲每一幀的角度

    while cap.isOpened():
        # 讀取影格
        ret, frame = cap.read()

        # 檢查是否成功讀取到影格
        if not ret:
            print("End of video. Exiting...")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # 取得右腳 hip、knee、ankle 的 3D 座標
            right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])

            right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z])

            right_ankle = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z])

            # 計算夾角（以弧度為單位）
            vector1 = right_hip - right_knee
            vector2 = right_ankle - right_knee
            dot_product = np.dot(vector1, vector2)
            norm_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
            cos_theta = dot_product / norm_product
            angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))

            # 將弧度轉換為角度
            angle_deg = np.degrees(angle_rad)

            # 將角度添加到列表中
            angles_list.append(angle_deg)

            # 設定閾值
            threshold_high = 110
            threshold_low = 50

            # 判斷角度是否超過閾值
            if angle_deg > threshold_high:
                text_color = (0, 255, 0)  # 綠色
            elif angle_deg < threshold_low:
                text_color = (255, 165, 0)  # 橘色
            else:
                text_color = (255, 0, 0)  # 藍色

            # Visualize angle at the top-right corner with color based on threshold
            text_position = (image.shape[1] - 150, 30)  # 設定文字位置
            cv2.putText(image, f"Angle: {angle_deg:.2f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2, cv2.LINE_AA)

            # Curl counter logic
            if angle_deg > 110:
                stage = "stand"
            if angle_deg < 50 and stage == 'stand':
                stage = "sit"
                counter += 1
                print(counter)

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # 顯示處理後的影像
        cv2.imshow('Mediapipe Feed', image)

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                )
        
        cv2.imshow('Mediapipe Feed', image)

        # 按 'q' 鍵退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # 釋放影片檔案來源
    cap.release()
    cv2.destroyAllWindows()

# 繪製角度變化圖表
plt.plot(angles_list)
plt.title('Angle Changes Over Frames')
plt.xlabel('Frame Index')
plt.ylabel('Angle (degrees)')
plt.show()

# Output final counters
print(f"Final sit to stand: {counter}")

2.mov

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 使用影片檔案來源
cap = cv2.VideoCapture("/Users/peilinyang/Desktop/Sit_to_Stand/2.mov")

# Curl counter variables
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    angles_list = []  # 用來存儲每一幀的角度

    while cap.isOpened():
        # 讀取影格
        ret, frame = cap.read()

        # 檢查是否成功讀取到影格
        if not ret:
            print("End of video. Exiting...")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # 取得右腳 hip、knee、ankle 的 3D 座標
            right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])

            right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z])

            right_ankle = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z])

            # 計算夾角（以弧度為單位）
            vector1 = right_hip - right_knee
            vector2 = right_ankle - right_knee
            dot_product = np.dot(vector1, vector2)
            norm_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
            cos_theta = dot_product / norm_product
            angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))

            # 將弧度轉換為角度
            angle_deg = np.degrees(angle_rad)

            # 將角度添加到列表中
            angles_list.append(angle_deg)

            # 設定閾值
            threshold_high = 110
            threshold_low = 50

            # 判斷角度是否超過閾值
            if angle_deg > threshold_high:
                text_color = (0, 255, 0)  # 綠色
            elif angle_deg < threshold_low:
                text_color = (255, 165, 0)  # 橘色
            else:
                text_color = (255, 0, 0)  # 藍色

            # Visualize angle at the top-right corner with color based on threshold
            text_position = (image.shape[1] - 150, 30)  # 設定文字位置
            cv2.putText(image, f"Angle: {angle_deg:.2f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2, cv2.LINE_AA)

            # Curl counter logic
            if angle_deg > 110:
                stage = "stand"
            if angle_deg < 50 and stage == 'stand':
                stage = "sit"
                counter += 1
                print(counter)

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # 顯示處理後的影像
        cv2.imshow('Mediapipe Feed', image)

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                )
        
        cv2.imshow('Mediapipe Feed', image)

        # 按 'q' 鍵退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # 釋放影片檔案來源
    cap.release()
    cv2.destroyAllWindows()

# 繪製角度變化圖表
plt.plot(angles_list)
plt.title('Angle Changes Over Frames')
plt.xlabel('Frame Index')
plt.ylabel('Angle (degrees)')
plt.show()

# Output final counters
print(f"Final sit to stand: {counter}")

6.mov

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# 使用影片檔案來源
cap = cv2.VideoCapture("/Users/peilinyang/Desktop/Sit_to_Stand/6.mov")

# Curl counter variables
counter = 0
stage = None

# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    angles_list = []  # 用來存儲每一幀的角度

    while cap.isOpened():
        # 讀取影格
        ret, frame = cap.read()

        # 檢查是否成功讀取到影格
        if not ret:
            print("End of video. Exiting...")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # 取得右腳 hip、knee、ankle 的 3D 座標
            right_hip = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z])

            right_knee = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z])

            right_ankle = np.array([landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z])

            # 計算夾角（以弧度為單位）
            vector1 = right_hip - right_knee
            vector2 = right_ankle - right_knee
            dot_product = np.dot(vector1, vector2)
            norm_product = np.linalg.norm(vector1) * np.linalg.norm(vector2)
            cos_theta = dot_product / norm_product
            angle_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))

            # 將弧度轉換為角度
            angle_deg = np.degrees(angle_rad)

            # 將角度添加到列表中
            angles_list.append(angle_deg)

            # 設定閾值
            threshold_high = 110
            threshold_low = 50

            # 判斷角度是否超過閾值
            if angle_deg > threshold_high:
                text_color = (0, 255, 0)  # 綠色
            elif angle_deg < threshold_low:
                text_color = (255, 165, 0)  # 橘色
            else:
                text_color = (255, 0, 0)  # 藍色

            # Visualize angle at the top-right corner with color based on threshold
            text_position = (image.shape[1] - 150, 30)  # 設定文字位置
            cv2.putText(image, f"Angle: {angle_deg:.2f}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 2, cv2.LINE_AA)

            # Curl counter logic
            if angle_deg > 110:
                stage = "stand"
            if angle_deg < 50 and stage == 'stand':
                stage = "sit"
                counter += 1
                print(counter)

        except:
            pass

        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # 顯示處理後的影像
        cv2.imshow('Mediapipe Feed', image)

        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                )
        
        cv2.imshow('Mediapipe Feed', image)

        # 按 'q' 鍵退出
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # 釋放影片檔案來源
    cap.release()
    cv2.destroyAllWindows()

# 繪製角度變化圖表
plt.plot(angles_list)
plt.title('Angle Changes Over Frames')
plt.xlabel('Frame Index')
plt.ylabel('Angle (degrees)')
plt.show()

# Output final counters
print(f"Final sit to stand: {counter}")